# December 20, 2021

https://adventofcode.com/2021/day/20

In [21]:
import pandas as pd
import numpy as np
import datetime
import re

In [22]:
def pnow():
    print( datetime.datetime.now().isoformat() )

In [23]:
def format_data( text ):
    lines = text.split("\n")
    for i in range(len(lines)):
        lines[i] = re.sub("#", "1", lines[i])
        lines[i] = re.sub("\.", "0", lines[i])

    return {"decoder": lines[0], "image":lines[2:]}

In [24]:
with open("../data/2021/20.txt", "r") as f:
    data_str = f.read()
data = format_data( data_str )

In [70]:
test_str = '''..#.#..#####.#.#.#.###.##.....###.##.#..###.####..#####..#....#..#..##..###..######.###...####..#..#####..##..#.#####...##.#.#..#.##..#.#......#.###.######.###.####...#.##.##..#..#..#####.....#.#....###..#.##......#.....#..#..#..##..#...##.######.####.####.#.#...#.......#..#.#.#...####.##.#......#..#...##.#.##..#...##.#.##..###.#......#.#.......#.#.#.####.###.##...#.....####.#..#..#.##.#....##..#.####....##...##..#...#......#.#.......#.......##..####..#...#.#.#...##..#.#..###..#####........#..####......#..#

#..#.\n#....\n##..#\n..#..\n..###'''
test = format_data( test_str )
test

{'decoder': '00101001111101010101110110000011101101001110111100111110010000100100110011100111111011100011110010011111001100101111100011010100101100101000000101110111111011101111000101101100100100111110000010100001110010110000001000001001001001100100011011111101111011110101000100000001001010100011110110100000010010001101011001000110101100111010000001010000000101010111101110110001000001111010010010110100001100101111000011000110010001000000101000000010000000110011110010001010100011001010011100111110000000010011110000001001',
 'image': ['10010', '10000', '11001', '00100', '00111']}

# Part 1

In [81]:
def binary( x ):
    '''convert string of 1s and 0s to decimal number'''
    return sum( [int(d) * 2**(len(x)-1-i) for i,d in enumerate(x)] )

def pad_image( image, times=1 ):
    # Add a border of 0s around the image
    # times = number of times to add consecutive border
    
    wid = len(image[0])

    def _blank_():
        # a blank line with additional padding on left and right
        return "0"*(times*2+wid)
    
    padded_image = []
    for i in range(times):
        padded_image.append( _blank_() )
    padded_image += ["0"*times + line + "0"*times for line in image ]
    for i in range(times):
        padded_image.append( _blank_() )
    return padded_image

def enhance( image, line, char, decoder ):
    section = image[line-1][char-1:char+2] + image[line][char-1:char+2] + image[line+1][char-1:char+2]
    key = binary(section)
    return decoder[key]

def count_ones( image ):
    return sum([ sum([int(c) for c in line]) for line in image] )

def print_image( image, to_print=None ):
    print("\n".join( ["".join([c for c in line]) for line in image] ))

def infinity_chart(t, decoder):
    # determine state of pixels in the "infinite" border, which are far away from the image itself.
    if decoder[0] == "1":
        # case: border turns to 1 and stays there
        if decoder[511] == "1":
            return "1" if t > 0 else "0"
        # case: border alternates between 1 and 0
        if decoder[511] == "0":
            return str(t%2)
        
    # case: border stays 0
    return "0"

def border_line( ch, wid ):
    return ch*wid

def convolve(image, decoder, times=1, show_progress=True):
    new_image = pad_image( image, times=times+1 )

    oht = len(image)

    # get corners of original image
    # coords are line, char
    inset = (times+1)

    if show_progress:
        print_image(new_image)
        print("\n")
    else:
        pnow()

    for t in range(1, times+1):
        print("iteration", t)
        
        start = inset-t   # calculations start on this row and this column (0-indexed)
        end = inset + oht + t - 1 # calculations end on this row and this column (0-indexed)
        infty = infinity_chart(t, decoder)

        # start with blank lines
        next_image = []

        for i in range(start):
            # outside border alternates between 0 and 1 for this puzzle input
            next_image.append(border_line(infty, len(new_image[0])))

        for i in range(start, end+1):
            next_image.append( infty*start + "".join( [enhance(new_image, i, j, decoder) for j in range(start, end+1) ] ) + infty*start )

        for i in range(start):
            # outside border alternates between 0 and 1 for this puzzle input
            next_image.append(border_line(infty, len(new_image[0])))

        new_image = next_image
        if show_progress:
            print_image(new_image)
            print("\n")
        else:
            pnow()

    return new_image


In [82]:
out = convolve( test["image"], test["decoder"],2 )

00000000000
00000000000
00000000000
00010010000
00010000000
00011001000
00000100000
00000111000
00000000000
00000000000
00000000000


iteration 1
00000000000
00000000000
00011011000
00100101000
00110100100
00111100100
00010011000
00001100100
00000101000
00000000000
00000000000


iteration 2
00000000000
00000000100
00100101000
01010001110
01000110100
01000001010
00101111100
00010111110
00001101100
00000111000
00000000000




In [83]:
count_ones(out)

35

In [84]:
# 5363 too high
# 5291 too high
out = convolve( data["image"], data["decoder"], 2, show_progress=False )
count_ones( out )

2025-04-02T21:10:23.977831
iteration 1
2025-04-02T21:10:23.995720
iteration 2
2025-04-02T21:10:24.014422


5275

# Part 2

In [85]:
out = convolve( test["image"], test["decoder"], 50, show_progress = False )

2025-04-02T21:10:32.541209
iteration 1
2025-04-02T21:10:32.541209
iteration 2
2025-04-02T21:10:32.541209
iteration 3
2025-04-02T21:10:32.542206
iteration 4
2025-04-02T21:10:32.542206
iteration 5
2025-04-02T21:10:32.542206
iteration 6
2025-04-02T21:10:32.543260
iteration 7
2025-04-02T21:10:32.544280
iteration 8
2025-04-02T21:10:32.544280
iteration 9
2025-04-02T21:10:32.545344
iteration 10
2025-04-02T21:10:32.546345
iteration 11
2025-04-02T21:10:32.548343
iteration 12
2025-04-02T21:10:32.549342
iteration 13
2025-04-02T21:10:32.551436
iteration 14
2025-04-02T21:10:32.553431
iteration 15
2025-04-02T21:10:32.555425
iteration 16
2025-04-02T21:10:32.558414
iteration 17
2025-04-02T21:10:32.560714
iteration 18
2025-04-02T21:10:32.564184
iteration 19
2025-04-02T21:10:32.566973
iteration 20
2025-04-02T21:10:32.571040
iteration 21
2025-04-02T21:10:32.575638
iteration 22
2025-04-02T21:10:32.579668
iteration 23
2025-04-02T21:10:32.583663
iteration 24
2025-04-02T21:10:32.588703
iteration 25
2025-04-0

In [86]:
count_ones( out )

3351

In [87]:
out = convolve( data["image"], data["decoder"], 50, show_progress=False )

2025-04-02T21:10:47.422232
iteration 1
2025-04-02T21:10:47.442501
iteration 2
2025-04-02T21:10:47.473727
iteration 3
2025-04-02T21:10:47.497155
iteration 4
2025-04-02T21:10:47.518085
iteration 5
2025-04-02T21:10:47.538019
iteration 6
2025-04-02T21:10:47.559952
iteration 7
2025-04-02T21:10:47.580957
iteration 8
2025-04-02T21:10:47.602968
iteration 9
2025-04-02T21:10:47.625807
iteration 10
2025-04-02T21:10:47.650323
iteration 11
2025-04-02T21:10:47.675808
iteration 12
2025-04-02T21:10:47.700116
iteration 13
2025-04-02T21:10:47.726888
iteration 14
2025-04-02T21:10:47.755638
iteration 15
2025-04-02T21:10:47.784250
iteration 16
2025-04-02T21:10:47.815661
iteration 17
2025-04-02T21:10:47.844674
iteration 18
2025-04-02T21:10:47.875492
iteration 19
2025-04-02T21:10:47.906531
iteration 20
2025-04-02T21:10:47.941082
iteration 21
2025-04-02T21:10:47.974882
iteration 22
2025-04-02T21:10:48.012231
iteration 23
2025-04-02T21:10:48.048295
iteration 24
2025-04-02T21:10:48.085703
iteration 25
2025-04-0

In [88]:
count_ones(out)

16482